# BEAT-PD Challenge

Challenge website : https://www.synapse.org/#!Synapse:syn20825169/wiki/596118

Data information : https://www.synapse.org/#!Synapse:syn20825169/wiki/600405


### Ideas/Doubts [Laureano]

VAD like thing to remove unwanted data?
modified MFCC?
X,Y,Z = relative positions or acceleration?

Imp: Predict per person. Maybe UBM like thing and adapt it

In [13]:
# Import required libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML, display

# Imports for the high pass signal
from scipy.signal import butter, freqz, lfilter

# KFold
from sklearn.model_selection import KFold

# Import required modules
from sklearn.preprocessing import StandardScaler

import os.path

# To write WAV File
from scipy.io.wavfile import write

# To make derivative work on multiple CPUs
from concurrent.futures import ProcessPoolExecutor
from functools import partial

import sys

from sklearn.model_selection import StratifiedKFold

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# import transform_data
from transform_data import *
from create_graphs import *


In [15]:
# Data paths
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"

# FIXME : Move this to data?
path_save_accelerometer_plots = "/home/sjoshi/codes/python/BeatPD/code/accelerometer_plots/"

possible to have participant characteristics from additional db data? ex https://ieeexplore.ieee.org/abstract/document/7911257

# CIS-PD Database

Contains 16 subject_id (patients) for the training set 

- Gender: 11 Male, 5 Female 
- Race: 15 White, 1 NA
- Ethnicity: 15 Not Hispanic or Latino, 1 Unknown
- Age average (standard deviation) : 62.8125 (10.8579)

### CIS-PD: Basic accelerometer visualization

In [ ]:
data_type = "cis"

# TODO: explain
path_train_data, df_train_label = define_data_type(data_type=data_type, data_dir=data_dir)
# display(df_train_label)
# List of interesting measurement id we want to look at
# list_measurement_id=[#'ab5287f4-8261-47ad-8ff2-22b5fe5d246e',
#'db2e053a-0fb8-4206-891a-6f079fb14e3a']#,
# 'ef5b1267-c212-46c5-aab0-4f4437bc6c67',
# '4ec74fb9-7347-435d-83dc-79ad74c3bc49',
# '8e8539ad-8841-476b-b15c-888ce3461989',
# '22b88456-fe8f-4138-af55-be12afca4b81',
# 'ad84583d-e5ae-4926-b077-531a0f7d08a9',
# 'eef56825-940a-4c3e-aebb-60838d60869e',
# 'e0441156-c4b8-467c-8f4f-3b532d594d8f',
# '464ac314-6c4b-4c4a-957c-28a2339150d6']

list_measurement_id = [
    "5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a",
    "cc7b822c-e310-46f0-a8ea-98c95fdb67a1",
    "5163afe8-a6b0-4ea4-b2ba-9b4501dd5912",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "2e3a4c9c-ff01-4a28-bfcf-ce9b7633a39d",  # no inactivity should be removed
    "3cf49c01-0499-4bad-9167-67691711204a",  # no inactivity should be removed PAS LA??
    "3d0f965c-9d72-43d1-9369-1ea3acf963cc",  # PAS LA ???
    "4b269cc2-8f0c-4816-adbf-10c0069b8833",
    "4bc51b90-bfce-4231-85e1-5de3b4bc0745",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
]  # bit of inactivity in the middle]

list_measurement_id = [
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
]

# Filter df_train_label according to the measurement_id we are most interested in
df_train_label = interesting_patients(
    df_train_label=df_train_label, list_measurement_id=list_measurement_id
)

# Display filtered df_train_label
display(df_train_label)

# path_no_inactivity_data = remove_inactivity_pct_change(df_train_label)

# Plot the accelerometer data
plot_accelerometer(df_train_label,
                   data_type=data_type,
                   path_train_data=path_train_data,
                   path_accelerometer_plots=path_save_accelerometer_plots
)

### CIS-PD: Create Masks for inactivity removal 

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)

print(path_train_data)

In [1]:
data_type = "cis"

remove_inactivity_highpass(
    df_train_label,
    path_train_data,
    data_type,
    energy_threshold=5,
    duration_threshold=3000,
    plot_frequency_response=False,
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.'+
    data_subset+'.high_pass_mask/')

NameError: name 'remove_inactivity_highpass' is not defined

### CIS-PD: Create first derivative

In [2]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)


do_work = partial(
    get_first_derivative, 
    path_train_data=path_train_data,
    derivative_path=data_dir+"cis-pd.training_data.derivative_original_data/",
    padding=True, 
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.training_data.high_pass_mask/',
)

num_jobs = 1
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

NameError: name 'define_data_type' is not defined

### CIS-PD: Create WAV files - Training Data

#### For Y

In [ ]:
data_type = "cis"
data_subset='training_data'

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)
do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Y/',
    sAxis='Y',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 8
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### CIS-PD Write Wav Ancillary Data Y

In [4]:
data_type = "cis"
data_subset='ancillary_data' 

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)
do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Y/',
    sAxis='Y',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 6
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/10005cf2-85cf-47ad-8441-f15fb1509f69.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/f25cfad6-1835-4f78-9192-d28581ffa4d8.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/10005cf2-85cf-47ad-8441-f15fb1509f69.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e9fa3c88-3eba-45a6-93c7-3ef4e5c8f846.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/f55eac42-c808-447b-9b9b-02c10cee579f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/541740be-1158-4169-becb-5ca4926ecd0c.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5cf1a9ff-b13c-457b-a661-ccd617842027.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/78428c75-bb3f-48c9-87f1-

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/406135de-3156-4f03-b38d-7758bf8d58bb.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1bb07775-5199-4fca-90e7-28038a3dd8f1.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/27770f06-a85b-483c-a793-60f7c5f5167e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/42ebe42f-ce33-4633-aa65-575472ff0c99.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/860cd080-5f6f-45ac-a1cc-f791b13e6555.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5a9b01ef-2deb-4a98-96fe-5797822334fe.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/c34dc766-74a2-46b4-82a3-a218303ffb26.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1bb07775-5199-4fca-90e7

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/39c50dd4-4433-4ead-9c97-fddfb96974f7.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fd46a3a4-965f-4e8e-a957-7aea9fa8d3de.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/593174b5-8802-41e5-84bf-713da62667b9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/d3a1eb8d-e993-4b41-9a28-2f1205d0e08c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/18ae03ad-e265-4d50-bd50-6baad42007fb.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/aa4a923e-d509-4abd-9c0a-4a6c637e37de.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/593174b5-8802-41e5-84bf-713da62667b9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fd46a3a4-965f-4e8e-

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/4b9d0dc6-e291-4a8f-a838-750038e789cf.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e2b029c2-af71-406c-8232-116cf1293ecd.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/46b51fb7-b901-43cd-b5f8-d1c9aef22e37.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/c7b935d5-11e3-4ddd-a6f5-e2d507cdee9b.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a7015551-83fe-453c-9452-6d7fef9a2a26.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/eab28778-e4db-462d-baf2-30ebdcd2ad7e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/c014e4c6-68b5-48ba-b817-2317b93f8ea4.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a7015551-83fe-453c-94

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/96f949fa-febc-4175-b654-9973e68bb408.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2e905c74-b895-4b0d-a6ca-9274f8cee767.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/0bb8a566-fbf5-4710-a4cd-2524a2a328e3.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fba2201b-50bb-4f64-a4a2-71735ce1a589.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a0d4fa63-827d-4c97-affd-bfa4e2dce425.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a29a468d-1db0-4a3a-a677-f81786fbef87.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/96f949fa-febc-4175-b654-9973e68bb408.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/b3e3dce2-5c37-4d09-8

...

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/170303da-330c-44dd-88df-d8c59148bfb2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1a3b8150-bfb4-4063-a084-55252ee3c458.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/377bc4a0-8da1-49f9-8a2d-283a02f571bc.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/85179480-bbc9-4241-8d08-4b8a8fb644f2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/786296cd-872e-431f-a851-8c7691d0d176.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/96428e26-10ed-421e-b853-8eccd5a07961.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/170303da-330c-44dd-88df-d8c59148bfb2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/85179480-bbc9-4241-

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e92bcd74-81cf-4a0c-b11f-c2f7c5647315.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/aaab722d-32e2-4fda-a428-9e8365074d84.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/bf67f14e-5ecc-4b46-94b9-c242a5e7d7d1.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/8cb01d26-2ab0-4ed6-a686-41b2bd4a00c5.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/f4db1429-1632-45fa-b5d2-571e1c79aaf7.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/6fb8093d-5b26-4580-a4f8-0c7ded523f01.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/9be6f67c-4a95-4814-bf28-b0c7d1921012.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/d79e42ba-0a21-4afd-9

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fee641d0-3c0f-4de4-88e8-a9512eb228ac.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/41a599c3-3325-4f3d-87cc-e5e63fa7ff5e.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/93b9c431-e7a9-48e5-aea6-0b05f1b08eea.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2d073ae5-33f0-412a-b526-0ab42337355c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/26636484-5c37-4e59-9158-5ff11c58dcae.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fee641d0-3c0f-4de4-88e8-a9512eb228ac.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/7a267ca8-d47d-4932-bec6-3ae53f68a7aa.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/55753704-38d4-46a7-b

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1286c4d8-5e44-4ede-aabc-7b8662e2156a.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e1586f1b-d51b-4468-b21e-d951c0b3fa29.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2eb0b592-b038-4c15-931d-b59cc97a8975.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/74485ef6-0db6-4b06-8941-f55f624fe569.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/074c5992-3ef8-4187-a895-f8650f8e245f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/57c3696d-eaec-4d34-b8ae-2addf6d6fa78.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2eb0b592-b038-4c15-931d-b59cc97a8975.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/61afcf16-a7c5-490d-9562

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/132b839a-5383-446f-8120-a439e18cae47.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5d0ea615-e03f-4d1c-9772-e7e83e5f40ef.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/db80e345-ad48-43d6-b7a9-4d2518e9831d.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/310a15c9-1863-4524-904e-2dc730d58329.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a1612a58-a5e1-43bb-8f31-0cbd97a5d806.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/63c0aeb7-0d28-4dd4-b80b-58e158b99016.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/edcb47d7-33d9-4fe3-be2d-9afd08b90fc5.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5d0ea615-e03f-4d1c-977

#### For Z

In [ ]:
data_type = "cis"
data_subset='training_data'

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset) 

do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Z/',
    sAxis='Z',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 8
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### CIS-PD Write Wav Ancillary Data Z

In [ ]:
data_type = "cis"
data_subset='ancillary_data' 

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)

do_work = partial(
    write_wav,
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Z/',
    sAxis='Z',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 6
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

## KFold 

In [21]:
def get_k_fold(df_train_label,
               n_splits=5,
               subject_id=None,
               data_subset='training_data',
               data_real_subtype=""):
    """
    Function that returns a list of X dataframes (X is according to the number of n_splits chosen)

    The dataframes are the labels needed according to the split 

    Keyword Arguments:
    - df_train_label: Dataframe containing the labels
    - n_split: Optional. The number of folds. Default: 5
    - subject_id: Optional. Specify a subject_id to get measurement_id only for that subject_id 
    - data_real_subtype: Only for REAL-PD database
    """
    kf = StratifiedKFold(n_splits) # shuffle??? 

    # Building the dataframe to split
    X = []

    # if we want the data split for one specific subject_id
    if subject_id:
        df_train_label_subject_id = df_train_label.groupby("subject_id")
        X = df_train_label_subject_id.get_group(subject_id)
    else:
        # if we want to have all a split for all data no matter the subject_id
        # NOTE: I didn't make sure to have one subject_id represented in both train/test
        for idx in df_train_label.index:
            X.append([df_train_label["measurement_id"][idx]])
        X = pd.DataFrame(X)

    kf.get_n_splits(X)

    # Building lists of df_train_label because we have by default 5 splits,
    # so the lists will contain 5 DataFrames with different split indices required
    list_df_train_label = list()
    list_df_test_label = list()
    split_idx = 0
    
    # Removing NaN values only for the purpose of kf.split as stratified kfold don't like NaN values
#     print(type(X))
#     print('Before contains NaN ? : ', X.isnull().sum())
    X = prepro_missing_values(X)
#     print('After contains NaN ? : ', X_no_nan.isnull().sum())
    
    if subject_id == 1046:
        # 1046 is balanced on tremor because that's the only labels we have for that patient
        y = X.iloc[:,-1]
    else:
        # We do the KFOLDs on a balanced on/off only 
        y = X.iloc[:,-3] 

    print("----- " + str(subject_id) + " -----")
    for train_index, test_index in kf.split(X, y):
        print(len(X))
        print(len(df_train_label))
        df_train_label = X.iloc[train_index]
        df_test_label = X.iloc[test_index]
        
        # Following is just to see if the splits are balanced or not 
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        train_0, train_1, train_2, train_3, train_4 = len(train_y[train_y==0]), \
                                                        len(train_y[train_y==1]), \
                                                        len(train_y[train_y==2]), \
                                                        len(train_y[train_y==3]), \
                                                        len(train_y[train_y==4])
        test_0, test_1, test_2, test_3, test_4  = len(test_y[test_y==0]), \
                                                  len(test_y[test_y==1]), \
                                                  len(test_y[test_y==2]), \
                                                  len(test_y[test_y==3]), \
                                                  len(test_y[test_y==4])
        print('>Train: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d Test: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d' % (train_0, train_1, train_2, train_3, train_4, \
                                                        test_0, test_1, test_2, test_3, test_4))
        
        list_df_train_label.append(df_train_label)
        list_df_test_label.append(df_test_label)

        # name of the file according to its database and type
        # NOTE: Be careful that the end of the name of the folder where to save the kfolds is hardcoded here
        path_save_k_fold_dataframes = (
            data_dir + data_type + "-pd."+data_subset+".k_fold_v2/" + data_real_subtype + "/"
        )
        df_train_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_train_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        df_test_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_test_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        split_idx = split_idx + 1
    return list_df_train_label, list_df_test_label

### Create the K-Fold files for the CIS database 

In [ ]:
# Define the data type as we have two databases
data_type = "cis"
path_train_data, df_train_label = define_data_type(data_type=data_type)

# Group data by subject_id
df_train_label_subject_id = df_train_label.groupby("subject_id")

data_subset = 'training_data'

# Go through the subject_id and k-fold their data
# FIXME: get_k_fold could me renamed to just create the folds, save them, not return anything
for subject_id, value in df_train_label_subject_id:
    list_df_train_label, list_df_test_label = get_k_fold(
        df_train_label=df_train_label,
        n_splits=5,
        training_or_ancillary=data_subset,
        subject_id=subject_id
    )


### Create the K-Fold Files for the REAL database

In [23]:
# To generate the files, you have to uncomment one data_real_subtype at a time and
# execute this cell 3 times for the 3 subtypes.

data_type = "real"
data_subset='training_data' # Either training_data or ancillary_data 

for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    
    path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset,
                                                   data_real_subtype)

    # Group data by subject_id
    df_train_label_subject_id = df_train_label.groupby("subject_id")

    # Go through the subject_id and k-fold their data
    for subject_id, value in df_train_label_subject_id:
        list_df_train_label, list_df_test_label = get_k_fold(
            df_train_label=df_train_label,
            n_splits=5,
            subject_id=subject_id,
            data_subset=data_subset,
            data_real_subtype=data_real_subtype,
        )

data_dir :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/
data_type :  real
data_subset :  training_data
data_real_subtype : smartphone_accelerometer
----- hbv012 -----
42
591
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
42
33
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
42
33
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
42
34
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
42
34
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
----- hbv013 -----


/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


68
591
>Train: 0=34, 1=17, 2=0, 3=0, 4=0 Test: 0=9, 1=5, 2=0, 3=0, 4=0
68
53
>Train: 0=34, 1=17, 2=0, 3=0, 4=0 Test: 0=9, 1=5, 2=0, 3=0, 4=0
68
53
>Train: 0=34, 1=18, 2=0, 3=0, 4=0 Test: 0=9, 1=4, 2=0, 3=0, 4=0
68
54
>Train: 0=35, 1=18, 2=0, 3=0, 4=0 Test: 0=8, 1=4, 2=0, 3=0, 4=0
68
56
>Train: 0=35, 1=18, 2=0, 3=0, 4=0 Test: 0=8, 1=4, 2=0, 3=0, 4=0
----- hbv014 -----
48
591
>Train: 0=26, 1=10, 2=0, 3=0, 4=0 Test: 0=7, 1=3, 2=0, 3=0, 4=0
48
37
>Train: 0=26, 1=10, 2=0, 3=0, 4=0 Test: 0=7, 1=3, 2=0, 3=0, 4=0
48
37
>Train: 0=26, 1=10, 2=0, 3=0, 4=0 Test: 0=7, 1=3, 2=0, 3=0, 4=0


/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


48
38
>Train: 0=27, 1=11, 2=0, 3=0, 4=0 Test: 0=6, 1=2, 2=0, 3=0, 4=0
48
40
>Train: 0=27, 1=11, 2=0, 3=0, 4=0 Test: 0=6, 1=2, 2=0, 3=0, 4=0
----- hbv017 -----
69
591
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
----- hbv018 -----
35
591
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
35
28
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
35
28
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
35
28
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
35
28
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
----- hbv022 -----
57
591
>Train: 0=36, 1=7, 2=0, 3=0, 4=0 Test: 0=10, 1=2, 2=0, 3=0, 4=0
57
44
>Train: 0=37, 1=7, 2=0, 3=0, 4=0 Test: 0=9, 

...

/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


68
56
>Train: 0=35, 1=18, 2=0, 3=0, 4=0 Test: 0=8, 1=4, 2=0, 3=0, 4=0
----- hbv014 -----
48
591
>Train: 0=26, 1=10, 2=0, 3=0, 4=0 Test: 0=7, 1=3, 2=0, 3=0, 4=0
48
37
>Train: 0=26, 1=10, 2=0, 3=0, 4=0 Test: 0=7, 1=3, 2=0, 3=0, 4=0
48
37
>Train: 0=26, 1=10, 2=0, 3=0, 4=0 Test: 0=7, 1=3, 2=0, 3=0, 4=0
48
38
>Train: 0=27, 1=11, 2=0, 3=0, 4=0 Test: 0=6, 1=2, 2=0, 3=0, 4=0
48
40
>Train: 0=27, 1=11, 2=0, 3=0, 4=0 Test: 0=6, 1=2, 2=0, 3=0, 4=0
----- hbv017 -----
69
591
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
69
55
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
----- hbv018 -----
35
591
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
35
28
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
35
28
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 

/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


----- hbv023 -----
46
591
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
46
36
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
46
37
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
46
37
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
46
37
>Train: 0=0, 1=0, 2=0, 3=0, 4=0 Test: 0=0, 1=0, 2=0, 3=0, 4=0
----- hbv038 -----
44
591
>Train: 0=17, 1=17, 2=0, 3=0, 4=0 Test: 0=5, 1=5, 2=0, 3=0, 4=0
44
34
>Train: 0=17, 1=17, 2=0, 3=0, 4=0 Test: 0=5, 1=5, 2=0, 3=0, 4=0
44
34
>Train: 0=18, 1=18, 2=0, 3=0, 4=0 Test: 0=4, 1=4, 2=0, 3=0, 4=0
44
36
>Train: 0=18, 1=18, 2=0, 3=0, 4=0 Test: 0=4, 1=4, 2=0, 3=0, 4=0
44
36
>Train: 0=18, 1=18, 2=0, 3=0, 4=0 Test: 0=4, 1=4, 2=0, 3=0, 4=0
----- hbv043 -----
37
591
>Train: 0=13, 1=16, 2=0, 3=0, 4=0 Test: 0=4, 1=4, 2=0, 3=0, 4=0
37
29
>Train: 0=13, 1=16, 2=0, 3=0, 4=0 Test: 0=4, 1=4, 2=0, 3=0, 4=0
37
29
>Train: 0=14, 1=16, 2=0, 3=0, 4=0 Test: 0=3, 1=4, 2=0, 3=0, 4=0
37
30
>Train: 0=14, 1=16, 2=0, 3=0, 4=0 

/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/mpgill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


# REAL-PD Database

This database, originally named "Parkinson@Home" is renamed to "Real-PD" for this challenge. The study was made over 2 weeks, with at home monitoring. 

The devices used are an android phone, a motorolla watch. 
- `smartwatch_accelerometer` and `smartwatch_gyroscope` : Motorolla Watch
- `smartphone_accelerometer` : Android phone 

-> Question: so is the smartwatch & smartphone accelerometer should measure the same movements? 

The REAL-PD database has many missing values. 

The subject_id `hbv013` is the only one without missing data. Other patients all have at least one missing symptom (`diskenisia`, ) or two (`on/off and tremor`, `on_off and dyskinesia`, `dyskinesia and tremor`) missing.

Measurements id with no data (`on_off`, `dyskinesia` and `tremor` are all missing):
- `b50d1b0c-2cd1-45f8-9097-0742e5cbbcc8`
- `b598c177-4e38-4ea8-8543-bd8f7e580f96`
- `cf841bf8-0082-4ea3-999f-1f43e39a8dc6`
- `b1e15f8a-109f-459b-ba87-46899240ee66`
- `6f0e2580-56ec-4743-9356-d3e4d9a0aee5`
- `773536f6-9b70-43d0-b099-5d167d74924a`
- `54a0e841-ad45-4ba7-ac83-1785c5f7748b`
- `cd9ed2e2-7e04-44c7-b041-7788f133c193`
- `a6954a91-338b-4523-9e4a-5e69a8fac206`

The 3 symptoms are reported as follows in this dataset: 
- `on_off = {0,1}`
  - `Off` : 0 (medication is wearing off) 
  - `On` : 1 (medication is working)
  
- `dyskinesia = {0,1,2}`
  - Without dyskinesia: 0 
  - Non-troublesome dyskinesia: 1 
  - Severe dyskinesia: 2 
  
- `tremor = {0,1,2,3,4}` 
The description of the database mentions `tremor` is rated from 0 to 4 according to its severity, but from all the data, the maximum value of `tremor` recorded is 3. 

Data:
- ancillary
- clinical : UPDRS evaluation score 
- demographics : #TODO


### REAL-PD: Create Masks for inactivity removal for all subtypes

In [17]:
data_type = "real"
#data_real_subtype='smartphone_accelerometer'
#data_real_subtype = "smartwatch_accelerometer"
#data_real_subtype='smartwatch_gyroscope'

# List of interesting measurement id we want to look at
# list_measurement_id = [
#     "5b4c7c81-659d-40ea-a1fd-59622074fd10",
#     "ee053d95-c155-400d-ae42-fe24834ad4a9",
#     "ce51ee31-8553-4321-9f83-8cd3dabe2f66",
#     "e07708ff-7b8d-4070-af70-3aa81423ab5b",
#     #'7d3f4b7a-167f-4a26-9062-94ce9d8794c1',
#     "99af8d14-cd09-4107-9502-355378ba4e08",
#     #'7d5ac31a-cb53-40f7-8188-0b13724ea55c',
#     "9e43840b-dd89-498b-af1a-a62896a4d5d9",
#     "e391f546-bf8a-46c7-a16c-95bc02f40629",
# ]

# list_measurement_id = ['0c310608-1a32-4b09-b164-375d93ddb2aa']

# Filter df_train_label according to the measurement_id we are most interested in
# df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    data_subset='ancillary_data' #training_data testing_data 
    print(data_real_subtype)
    path_train_data, df_train_label = define_data_type(data_type,
                                                       data_dir,
                                                       data_subset,
                                                       data_real_subtype)

#     list_measurement_id = ['f465145e-562d-4c02-b347-bc42ae3b4998']#['33f5a031-43a8-496a-89ee-0b9d99019617', '2515fa8d-8320-4010-bcd9-5dba52b1b5ba']
    # Filter df_train_label according to the measurement_id we are most interested in
#     df_train_label = interesting_patients(df_train_label=df_train_label,
#                                           list_measurement_id=list_measurement_id)

    remove_inactivity_highpass(
        df_train_label,
        path_train_data=path_train_data,
        data_type=data_type,
        energy_threshold=5,
        duration_threshold=3000,
        plot_frequency_response=False,
        plot_accelerometer_after_removal=False,
        mask_path=data_dir+'/real-pd.'+
        data_subset+
        '.high_pass_mask/'+data_real_subtype+'/')

smartphone_accelerometer
data_dir :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/
data_type :  real
data_subset :  ancillary_data
data_real_subtype : smartphone_accelerometer
Working on  2515fa8d-8320-4010-bcd9-5dba52b1b5ba
Skipping 7fde941c-2d8a-4b33-b32d-af016161ee1e as it doesn't exist for the subtype
Skipping 92eea111-b6b0-4d07-a624-cf2b0ecea769 as it doesn't exist for the subtype
Skipping 3f5adb13-512c-4090-96ac-707703077845 as it doesn't exist for the subtype
Working on  42d34bdc-54a9-4b4f-bed6-c02219c6061f
Working on  3bc6b0d1-0204-4a3d-827a-1579a76b411e
Working on  803d4ad6-328d-494f-8923-329175c8b07f
Working on  f6dde26f-279c-4d83-9e5e-8b25cb7e31bd
Working on  ed9d2a14-cb01-42e6-85f0-180b70b2f821
Working on  83cdbcb3-034a-4eb2-9cc6-3757dd4aff22
Working on  2bb369bc-3300-4b73-a57e-58dccee47e07
Skipping 539ab11e-0d9f-4d42-a209-5261a3655275 as it doesn't exist for the subtype
Skipping 475759e6-bdad-44f4-a1be-63d18e475352 as it doesn't exist for the subtype
Working on  d4474cb5-c

Working on  79320b77-0112-47e6-90e5-020a07a3ffd4
Working on  09920c5d-0c9b-42e9-9b1c-9e853bd4fe50
Working on  f74fdd6f-d28e-4c38-af3d-2149802058ab
Working on  bdc5dc1d-7f8b-445c-8f19-16709dd53351
Working on  0d21a85d-a641-4069-ad2f-7b9d55b63635
Working on  b464bd85-cff0-466f-976f-9ffd29b798f8
Skipping f90d07fa-6362-4d6e-895a-03897208acab as it doesn't exist for the subtype
Working on  3af93fc4-3528-4fa9-8ad5-99f169d6491c
Skipping e479a780-b3fc-44de-b532-d4f3dc7e68fb as it doesn't exist for the subtype
Working on  7202317c-5623-4466-9de1-2042f111dee4
Working on  c7b4e3e2-ef34-4e84-95fa-f86aef445ddd
Working on  b7b33f54-22ed-4d38-88bd-41f31b8a2fdd
Working on  e8180e0a-4e1a-4708-8146-33eb729cf9f5
Skipping eee6e062-9d30-43d3-aa05-3f12f317c12e as it doesn't exist for the subtype
Skipping 2e079684-c0f3-4d3c-9db9-b440ac60e521 as it doesn't exist for the subtype
Working on  3aa9b290-6767-4a07-9693-7985e826371e
Working on  626147c2-5b14-404f-8ef4-59a61d989e8d
Working on  c6fb7516-3c9a-47e0-aed9

Skipping cde8e059-98d7-4864-83e7-692c2f3ba459 as it doesn't exist for the subtype
Working on  65557626-0075-4da9-b740-0e3e389a0df9
Working on  26f49660-ce1a-4946-8f83-f88850f03ec1
Skipping 4c942781-59a6-46e1-95e1-4d74bac4f72d as it doesn't exist for the subtype
Skipping 52cbf184-847c-4c40-8f7d-93abf9b7209d as it doesn't exist for the subtype
Working on  521d28af-2d12-4f47-a6eb-085030391136
Working on  e31c69c0-2820-47c6-afbe-f489e2809b3e
Working on  6545d215-9661-4cca-aceb-f59d52b577c9
Working on  51ecdb9d-bcb9-45fa-8df7-1ea7087e8494
Working on  ad6ac3dc-383a-47d7-8981-d3fa5023ac34
Working on  ecbe8a91-6976-481b-bd3e-501bb7691401
Working on  e40aacd3-0b44-4107-ac51-bd97462d81b2
Skipping 0f9f7bbf-84b2-4fb2-8c5f-e01f495c85b4 as it doesn't exist for the subtype
Working on  bd1b30e9-4aea-4a28-a627-4a7559a258d0
Working on  7909d932-3379-4653-9799-2e372c03daa3
Working on  c9706df3-8d80-479a-af0f-3093a50dea75
Skipping 34d52a27-7dd9-4a68-a989-94811d810839 as it doesn't exist for the subtype
Wo

Working on  523b86de-6271-46f7-9379-500e3e22f5df
Working on  9fa4862a-5b76-410e-afcf-73da4df8a8a5
Skipping 00ec8065-af5a-48be-98e6-a30dfe404747 as it doesn't exist for the subtype
Working on  3e402fec-91f4-46b4-b9e4-27f5c6f08f0e
Skipping dcabef2b-8ee7-4814-b6c3-73c9e5c91f1e as it doesn't exist for the subtype
Skipping 0ed06e7b-fd56-45cd-b287-913439a84b87 as it doesn't exist for the subtype
Skipping 8e741219-dab8-4efa-a8d4-630ec8565d0d as it doesn't exist for the subtype
Working on  6c94b7d8-9be0-4212-80a6-800436b1fd74
Skipping f24c8fcb-1b16-497f-ac12-a938235cc43c as it doesn't exist for the subtype
Skipping 3a18baf4-ddd8-48ad-8419-1a802eef4ecf as it doesn't exist for the subtype
Working on  80d3e287-4c41-4e6e-8ce6-9d1f74de9141
Working on  6f734c22-dc8f-4c99-95c1-d05889ed0746
Working on  8b66a621-86fc-4e55-9b12-c4373770b262
Skipping 5ac3c8f8-c1e0-451b-ba22-ec3c965a604e as it doesn't exist for the subtype
Skipping 627e7fba-82f4-4124-8b3e-9d875c0dbcf4 as it doesn't exist for the subtype
S

Working on  757b411a-6b2a-42fb-a806-4d89bd7d6fa2
Working on  7fcb1c8d-9dd8-4940-b54e-6ecd5082e110
Skipping d417f853-2455-4038-9086-fe7832ff9010 as it doesn't exist for the subtype
Working on  27b2058e-874e-4392-91b3-cde65928908f
Working on  a368d2c7-1bbe-413e-a463-dcc1735f3b16
Working on  13811568-56c3-4f62-9188-6f34fd972b03
Working on  aefbfe70-f94e-4393-99b6-3f326d98a11d
Working on  5d03d8e6-8a96-4de6-b3ef-94fc8fb07446
Working on  c886f69d-08bc-45a0-b281-00a08e3aaa4b
Working on  833251d3-ec8f-4aab-9b0b-f42d3cf3acd6
Working on  c3019cd6-6143-48ab-a1b5-a83225e6ee10
Skipping 086cfab2-8021-44d8-8e18-b8c9c873ad72 as it doesn't exist for the subtype
Skipping 481b49b5-18ab-49e1-9d54-2414f2bd285c as it doesn't exist for the subtype
Working on  d4ecca82-4141-41f5-9f63-ab0ea711f482
Working on  7b633895-3c46-4e9d-b5c8-caf52d3b77ad
Working on  db191542-0a2f-4cbb-893f-d41a57a67995
Skipping 63ea80d6-f89c-40dc-bcfc-8063eae2fef1 as it doesn't exist for the subtype
Working on  da639320-04b7-4782-9eee

Working on  a7b66513-8868-45ba-86be-dc7784d92b32
Working on  a6aed29d-bd5d-4cc4-b395-86fd5b9a3cbc
Working on  43fc02af-b801-4f21-8478-46095fde5f2b
Skipping 9f13e050-68c5-4c7a-944d-6ffc061547db as it doesn't exist for the subtype
Working on  55b8a2b3-d06b-4cd6-afe5-2f5d244ecfdf
Working on  3f273747-3548-4e4d-93e9-814f5ea76e7b
Skipping e0a00b7f-58a8-4a13-bc6e-1ab4df2e9bb2 as it doesn't exist for the subtype
Working on  c62cb96c-02d1-4875-8afd-dc89aed6e1cb
Working on  59ca1758-6e94-4d7f-8b39-a9f7e8d445fd
Working on  520f8ac9-787c-47e9-ad35-ab72afef5cbe
Skipping fa47e8b1-b8de-434b-b8fe-babe9ecac82a as it doesn't exist for the subtype
Working on  e079ed52-0edb-4aac-ba6f-fef339e7b815
Working on  b8ebe51d-9889-48cc-9d6f-3ce670753ede
Working on  6b72ac93-410b-4743-890e-90222a162715
Working on  c332b7e7-d8fd-48c5-9baf-ad49f31ca45d
Working on  225b45a8-5e2c-4c12-8cec-08d8ece457cc
Skipping 00ff67dd-2bd7-41f7-acf6-ae809cc46120 as it doesn't exist for the subtype
Working on  90f22780-d114-48c3-9ccb

Working on  67b17ed4-530e-473a-b9c2-8328350d53a0
Working on  9588ee17-ba87-4f89-8fce-7be9032e0173
Working on  d1de0d41-67f2-420a-89bf-67625db6a202
Working on  76f62ef2-bd1a-43c1-8719-3d06d039393d
Working on  1a9131da-da3f-4946-86fd-1a1fe7c1b197
Working on  166ba983-209f-4639-a5a6-d6e66adeba2b
Working on  b76b8da8-a7a8-4fdb-adff-92b5e32c5290
Working on  4bc18809-1cac-4cb6-ab27-c5c1103f874e
Working on  a9134fea-009e-46a1-a8b4-08f020bfcf77
Skipping 3d247c75-ac8f-447d-8d98-9405ca2b6fc8 as it doesn't exist for the subtype
Working on  81001faa-de53-4b25-b9b2-a5baea335914
Working on  6f41582d-dbef-4600-b52d-488f50382364
Working on  4eb2d05f-4b8a-4525-8568-4d07c5b7d739
Working on  7e7a2e08-6e6c-4259-97d6-4d7e7ca99c72
Working on  7a1c1a27-7e08-432d-8701-0d392c691c80
Working on  6f9bf134-82d7-45f0-a3dc-3dbbeeb2b988
Working on  bd48f1aa-7d9b-4613-8766-8b4c422a6b73
Working on  5736fb16-8fcf-4425-93bf-e52c04b52726
Working on  63037fdb-b601-4109-9122-f7cab5bf0d74
Working on  f427e4a8-e8a8-4143-8db8-

Working on  caf3bb66-3d7e-47c2-8437-8d7a28d3d13b
Skipping d02209e2-ccb6-43ca-88a4-5053c51312f9 as it doesn't exist for the subtype
Skipping 7e5db015-a0ad-4898-af48-8dbb927aad28 as it doesn't exist for the subtype
Working on  9410e6a1-be87-4b03-be0d-fae812c52d91
Working on  e667c1c6-1657-4939-bbfd-c07305d4499e
Working on  16e7c7b7-81bf-4483-9fe5-f434c7ca1ef8
Working on  278622b7-49af-4730-8e5a-c7e9c91c49b9
Working on  76406c61-ffa2-43a7-9f0a-8e0e2524380e
Working on  f182e0d5-cd90-4e25-b2d1-5f950e2ec341
Working on  15a5b255-d6ea-455c-9a67-02ac71d3890e
Working on  4b8e9621-5dca-4511-a40f-980dec857c57
Working on  caeaea08-2477-4b5b-8d57-cf85043459c7
Skipping 80a009f0-1461-4dfb-b735-7748ba78d3f9 as it doesn't exist for the subtype
Skipping e8a35db7-db21-4a1f-a278-caad9cf481fb as it doesn't exist for the subtype
Working on  4c5e949d-4813-496d-b4f6-bb445f8a9646
Working on  aea54c3e-6327-4e47-8d77-90dd11032d6e
Skipping 44675ca1-13bf-4384-9dc5-ef24d3008f01 as it doesn't exist for the subtype
Wo

Working on  3bf7736a-295d-4d8d-ad34-5655aee2f6c0
Working on  ac614cee-6aa1-4bae-87cc-efa6d00f86d8
Working on  9b548bee-6d39-4fce-8924-7369bd20a669
Working on  d74eef31-5adb-4d36-81ef-a51c86d72577
Working on  ed9882f5-e061-4686-9350-086ef55dc78b
Working on  899d4b0e-c3b1-4018-939c-df2ad414f1a4
Working on  3382d679-6070-4513-acb6-8daf1b1e2472
Working on  65b4c582-1239-4e98-aabe-3905e0692537
Working on  cd936b97-8a7d-4df5-8de6-5850c61be65d
Working on  958f587b-9098-4be0-8bce-bf5d29908dbf
Skipping 9d74f5e1-241a-4f4a-bc6f-2779edf410cd as it doesn't exist for the subtype
Working on  fe8dccc1-956f-4541-94e0-85b546aaf464
Skipping b1a5fd6d-db9c-4870-a3c0-943e0656d112 as it doesn't exist for the subtype
Working on  d8115ac8-a4db-4ba1-b875-fef4232a8494
Working on  885b9fe2-07a8-4db0-bf54-25016bb07483
Working on  4cfabd70-263b-40df-a29a-46975de4d8d1
Working on  db6686e7-a746-465d-8d29-b411220a4e6b
Skipping 321f893b-00e6-4c55-b247-52cb1f828e37 as it doesn't exist for the subtype
Skipping d43197ed-1

Working on  6ce4e0c5-a143-4b88-b2da-55e800df3939
Working on  f6e2ff78-bca5-4a91-901a-4bbd1f7734f7
Working on  7a1a8afc-4556-42fb-b718-61ea58d8cccd
Working on  e2d0a6b9-562f-4a21-9155-59d7c2cae49c
Working on  394d9466-4da1-4591-b403-84e777d86d32
Skipping 1ce4107d-8975-44d5-b5fd-addb2a9d6bdb as it doesn't exist for the subtype
Working on  e35da0ca-3e4e-4dcc-ac5f-11616a8398e8
Working on  d8b761ab-d444-4635-b7b2-95f59b75b2f1
Working on  fd732b1f-7c4f-4832-8072-8cb7e2bee80a
Working on  f7e99ec2-2055-4b39-8264-30baa1d4a7d5
Working on  844bf1a3-5bd6-4d3d-b23c-339423d41767
Working on  811593e5-1f28-42c9-8faf-072ae7980e3e
Working on  3f108781-78a2-449a-8d74-ad3cb279684d
Working on  c6092fd7-c429-42ec-9fdf-18328e5342c3
Skipping 19762c1b-f33a-4f8e-b0e4-6b4f6d38f312 as it doesn't exist for the subtype
Working on  b1873d17-d5a1-4274-8541-c46d30e91e72
Working on  5759106c-ecfb-45bf-b1a0-a950313f580a
Working on  d3825747-ec0e-4f34-b124-a27773480724
Working on  67663d81-bc34-4c23-b008-b04e1a5b6811
Wor

### REAL-PD: Create first derivative for all subtypes 

In [ ]:
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    path_train_data, df_train_label = define_data_type(data_type=data_type)
    print(len(df_train_label))
    for idx in df_train_label.index:
        try:
#             print('where we get the file : ', path_train_data)
            df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
        except FileNotFoundError:
            print('Removing ' + df_train_label["measurement_id"][idx] +
                  ' as it doesn\'t exist for ' +
                  data_real_subtype)
            df_train_label = df_train_label.drop(idx)
        print(len(df_train_label))
    do_work = partial(
        get_first_derivative, 
        path_train_data=path_train_data,
        derivative_path="real-pd.training_data.derivative_original_data/"+data_real_subtype+'/',
        padding=True, 
        mask_path=data_dir+'/real-pd.training_data.high_pass_mask/'+data_real_subtype+'/'
    )

    num_jobs = 8
    with ProcessPoolExecutor(num_jobs) as ex:
        results = list(ex.map(do_work, df_train_label['measurement_id']))


### REAL-PD: Create WAV files for all subtypes

### X Axis

In [18]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="X", data_type="real")

# data_type ="real"
# #FIXME add the one missing when it's done running 
# #'smartphone_accelerometer', 'smartwatch_accelerometer', smartwatch_gyroscope
# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset = 'ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type, data_subset, data_real_subtype)
# #     list_mesurement_id=['33f5a031-43a8-496a-89ee-0b9d99019617']
#     # Filter df_train_label according to the measurement_id we are most interested in
# #     df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)

#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
        
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'real-pd.'+data_subset+'.wav_X/'+data_real_subtype+'/',
#         sAxis='X',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/'
#     )

#     num_jobs = 6
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

data_dir :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/
data_type :  real
data_subset :  ancillary_data
data_real_subtype : smartphone_accelerometer
Removing 7fde941c-2d8a-4b33-b32d-af016161ee1e as it doesn't exist for smartphone_accelerometer
Removing 92eea111-b6b0-4d07-a624-cf2b0ecea769 as it doesn't exist for smartphone_accelerometer
Removing 3f5adb13-512c-4090-96ac-707703077845 as it doesn't exist for smartphone_accelerometer
Removing 539ab11e-0d9f-4d42-a209-5261a3655275 as it doesn't exist for smartphone_accelerometer
Removing 475759e6-bdad-44f4-a1be-63d18e475352 as it doesn't exist for smartphone_accelerometer
Removing 6542d311-9d13-400a-8a3c-61f0d52ef6f5 as it doesn't exist for smartphone_accelerometer
Removing 41c392cb-3495-4138-a1b2-f056704f160c as it doesn't exist for smartphone_accelerometer
Removing eb5c7874-8961-4bac-b1cb-e35e4d715ca5 as it doesn't exist for smartphone_accelerometer
Removing 0b37bc64-905b-4234-af29-9760c9e16346 as it doesn't exist for smartphone_accelero

Removing 63f6f59e-cd68-4329-9b8f-be61e0c71f82 as it doesn't exist for smartphone_accelerometer
Removing b67a84d7-80d0-4b37-8812-6715ef6f5fbb as it doesn't exist for smartphone_accelerometer
Removing 537b5264-ea4c-4539-924a-4bf6e9eb69ba as it doesn't exist for smartphone_accelerometer
Removing cf49a3ae-8921-4264-99ee-a06d5f05575a as it doesn't exist for smartphone_accelerometer
Removing 99200aa0-4834-4f34-96d9-71c3c1b765b1 as it doesn't exist for smartphone_accelerometer
Removing f2a6831f-e712-4eb7-b150-ac44f520679c as it doesn't exist for smartphone_accelerometer
Removing fd3e05f6-8d9a-43c9-8a93-2231d341650d as it doesn't exist for smartphone_accelerometer
Removing 732aa52a-ce30-43d7-9fb0-647aeeb49957 as it doesn't exist for smartphone_accelerometer
Removing b09b5104-3a37-4819-b7f5-ba721d024722 as it doesn't exist for smartphone_accelerometer
Removing c0a5ca86-ac24-4d73-8623-c2923677c617 as it doesn't exist for smartphone_accelerometer
Removing f0911246-5bbd-4299-9ead-a60551b2ebfc as i

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/58ea7790-a93c-4a9f-81c0-4757f237346e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/a4bf0da9-7116-4b57-ada2-d4e6fa116253.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/4e26e20d-344b-4bbf-9dbf-4a389c73784d.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/d4474cb5-c1c2-497d-921e-3ec7abe02f86.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/cc73fbc0-f17c-4564-8895-c963a8a44097.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/de88da3c-eff1-4827-ae3b-31472b96186f.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_d

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/40bfdda0-4125-48eb-ab08-3238ebdf4bc8.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/05849b77-59c8-4032-9982-fba459c250da.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/9dda3848-0758-4889-81ac-7a2d4fee50ef.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/b0776189-ec32-49b3-8ca4-27c993419e87.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/d02209e2-ccb6-43ca-88a4-5053c51312f9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/caf3bb66-3d7e-47c2-8437-8d7a28d3d13b.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_dat

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/278622b7-49af-4730-8e5a-c7e9c91c49b9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/39462bad-7b92-4b0b-865a-3d3091da34d2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/89094c0a-76f0-4b6c-8ff7-e3d869f86232.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/e1fbf359-29fc-4320-97cb-2a4d1e131d5d.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/0db55f4b-d580-4e3c-a55c-8f76a5321af1.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartphone_accelerometer/757b411a-6b2a-42fb-a806-4d89bd7d6fa2.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_da

...

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/d8b761ab-d444-4635-b7b2-95f59b75b2f1.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/e35da0ca-3e4e-4dcc-ac5f-11616a8398e8.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/e2d0a6b9-562f-4a21-9155-59d7c2cae49c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/f6e2ff78-bca5-4a91-901a-4bbd1f7734f7.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/fd732b1f-7c4f-4832-8072-8cb7e2bee80a.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/f7e99ec2-2055-4b39-8264-30baa1d4a7d5.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyrosc

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/4bc18809-1cac-4cb6-ab27-c5c1103f874e.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/9588ee17-ba87-4f89-8fce-7be9032e0173.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/1a9131da-da3f-4946-86fd-1a1fe7c1b197.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/d1de0d41-67f2-420a-89bf-67625db6a202.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/76f62ef2-bd1a-43c1-8719-3d06d039393d.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/166ba983-209f-4639-a5a6-d6e66adeba2b.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gy

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/ab6d059d-4437-4c10-96e8-8445cad60813.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/732aa52a-ce30-43d7-9fb0-647aeeb49957.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/6b83a350-00a7-4a54-831d-80a88d8d8d93.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/b09b5104-3a37-4819-b7f5-ba721d024722.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/c0a5ca86-ac24-4d73-8623-c2923677c617.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/6b83a350-00a7-4a54-831d-80a88d8d8d93.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyro

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/6f734c22-dc8f-4c99-95c1-d05889ed0746.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/6c94b7d8-9be0-4212-80a6-800436b1fd74.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/627e7fba-82f4-4124-8b3e-9d875c0dbcf4.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/a0918410-828e-4039-a3bb-bfcc980132a5.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/1f80cd74-b3df-41c9-820a-04a36ca74ad9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/f7e2ee64-faea-4d05-bd02-815fe29aa6be.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyrosc

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/14a1418c-7d35-4c24-bc97-fde11398ebd0.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/a497162d-a211-4e93-b2dd-42315c17d1b0.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/cde6f7fe-4b91-475c-a9dd-cac4ba722112.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/8acb3d70-09ab-42b2-a4dd-8e2fa8d3ad0d.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/a68c3091-ae9e-4450-8c52-ea56c1ae78d9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscope/0b389dc2-8d2d-4e21-bda0-2c54bca79f25.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_X/smartwatch_gyroscop

### Y axis

In [19]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Y", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data'#'training_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)

#     do_work = partial(
#         write_wav,
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Y/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Y',
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

data_dir :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/
data_type :  real
data_subset :  ancillary_data
data_real_subtype : smartphone_accelerometer
Removing 7fde941c-2d8a-4b33-b32d-af016161ee1e as it doesn't exist for smartphone_accelerometer
Removing 92eea111-b6b0-4d07-a624-cf2b0ecea769 as it doesn't exist for smartphone_accelerometer
Removing 3f5adb13-512c-4090-96ac-707703077845 as it doesn't exist for smartphone_accelerometer
Removing 539ab11e-0d9f-4d42-a209-5261a3655275 as it doesn't exist for smartphone_accelerometer
Removing 475759e6-bdad-44f4-a1be-63d18e475352 as it doesn't exist for smartphone_accelerometer
Removing 6542d311-9d13-400a-8a3c-61f0d52ef6f5 as it doesn't exist for smartphone_accelerometer
Removing 41c392cb-3495-4138-a1b2-f056704f160c as it doesn't exist for smartphone_accelerometer
Removing eb5c7874-8961-4bac-b1cb-e35e4d715ca5 as it doesn't exist for smartphone_accelerometer
Removing 0b37bc64-905b-4234-af29-9760c9e16346 as it doesn't exist for smartphone_accelero

Removing 537b5264-ea4c-4539-924a-4bf6e9eb69ba as it doesn't exist for smartphone_accelerometer
Removing cf49a3ae-8921-4264-99ee-a06d5f05575a as it doesn't exist for smartphone_accelerometer
Removing 99200aa0-4834-4f34-96d9-71c3c1b765b1 as it doesn't exist for smartphone_accelerometer
Removing f2a6831f-e712-4eb7-b150-ac44f520679c as it doesn't exist for smartphone_accelerometer
Removing fd3e05f6-8d9a-43c9-8a93-2231d341650d as it doesn't exist for smartphone_accelerometer
Removing 732aa52a-ce30-43d7-9fb0-647aeeb49957 as it doesn't exist for smartphone_accelerometer
Removing b09b5104-3a37-4819-b7f5-ba721d024722 as it doesn't exist for smartphone_accelerometer
Removing c0a5ca86-ac24-4d73-8623-c2923677c617 as it doesn't exist for smartphone_accelerometer
Removing f0911246-5bbd-4299-9ead-a60551b2ebfc as it doesn't exist for smartphone_accelerometer
Removing 18cd67ad-b280-4520-9582-d38b4f8004f8 as it doesn't exist for smartphone_accelerometer
Removing 59a2afdb-b324-421d-a0ba-e1b08dcd0585 as i

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/4e26e20d-344b-4bbf-9dbf-4a389c73784d.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/58ea7790-a93c-4a9f-81c0-4757f237346e.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/a4bf0da9-7116-4b57-ada2-d4e6fa116253.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7d9e47b4-75dc-4721-befc-c0a7fc1f68f4.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/cc73fbc0-f17c-4564-8895-c963a8a44097.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/08daa9d4-7cd7-4b17-8032-29021e3c14ce.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_dat

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/9dda3848-0758-4889-81ac-7a2d4fee50ef.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/33f5a031-43a8-496a-89ee-0b9d99019617.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/b0776189-ec32-49b3-8ca4-27c993419e87.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/caf3bb66-3d7e-47c2-8437-8d7a28d3d13b.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/d02209e2-ccb6-43ca-88a4-5053c51312f9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7e5db015-a0ad-4898-af48-8dbb927aad28.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/0db55f4b-d580-4e3c-a55c-8f76a5321af1.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/39462bad-7b92-4b0b-865a-3d3091da34d2.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7fcb1c8d-9dd8-4940-b54e-6ecd5082e110.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/d417f853-2455-4038-9086-fe7832ff9010.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/757b411a-6b2a-42fb-a806-4d89bd7d6fa2.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartphone_accelerometer/27b2058e-874e-4392-91b3-cde65928908f.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_dat

...

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/f6e2ff78-bca5-4a91-901a-4bbd1f7734f7.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/e2d0a6b9-562f-4a21-9155-59d7c2cae49c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/6ce4e0c5-a143-4b88-b2da-55e800df3939.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/e35da0ca-3e4e-4dcc-ac5f-11616a8398e8.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/394d9466-4da1-4591-b403-84e777d86d32.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/7a1a8afc-4556-42fb-b718-61ea58d8cccd.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_g

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/67b17ed4-530e-473a-b9c2-8328350d53a0.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/76f62ef2-bd1a-43c1-8719-3d06d039393d.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/b76b8da8-a7a8-4fdb-adff-92b5e32c5290.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/4bc18809-1cac-4cb6-ab27-c5c1103f874e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/a9134fea-009e-46a1-a8b4-08f020bfcf77.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/81001faa-de53-4b25-b9b2-a5baea335914.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyrosc

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/b288f078-92bf-4907-ab75-6e594b92ecaf.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/fd3e05f6-8d9a-43c9-8a93-2231d341650d.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/5e0f0682-235d-468e-a25c-c45844263d97.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/f2a6831f-e712-4eb7-b150-ac44f520679c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/1872cee9-3d53-4e8d-8d95-2145dd4e436e.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/6b83a350-00a7-4a54-831d-80a88d8d8d93.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_g

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/1f80cd74-b3df-41c9-820a-04a36ca74ad9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/3a18baf4-ddd8-48ad-8419-1a802eef4ecf.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/a0918410-828e-4039-a3bb-bfcc980132a5.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/f7e2ee64-faea-4d05-bd02-815fe29aa6be.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/a640e7d3-225a-4f45-8ec5-3c3f30cecca3.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/2116825d-2ca7-4566-8fc5-c717f8ff53cc.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyrosco

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/a68c3091-ae9e-4450-8c52-ea56c1ae78d9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/4cae887a-787b-4a69-801b-501697ee2e47.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/4f61b6cd-c94f-443d-9545-f343abd12592.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/cde6f7fe-4b91-475c-a9dd-cac4ba722112.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/aa851f48-3033-4e09-be6c-a78d909617d3.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gyroscope/a497162d-a211-4e93-b2dd-42315c17d1b0.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Y/smartwatch_gy

### Z Axis

In [20]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Z", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartwatch_gyroscope']:#['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     display(df_train_label)
#     for idx in df_train_label.index:
#         try:
#             print(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
#     display(df_train_label)
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Z/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Z'
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))
        

data_dir :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/
data_type :  real
data_subset :  ancillary_data
data_real_subtype : smartphone_accelerometer
Removing 7fde941c-2d8a-4b33-b32d-af016161ee1e as it doesn't exist for smartphone_accelerometer
Removing 92eea111-b6b0-4d07-a624-cf2b0ecea769 as it doesn't exist for smartphone_accelerometer
Removing 3f5adb13-512c-4090-96ac-707703077845 as it doesn't exist for smartphone_accelerometer
Removing 539ab11e-0d9f-4d42-a209-5261a3655275 as it doesn't exist for smartphone_accelerometer
Removing 475759e6-bdad-44f4-a1be-63d18e475352 as it doesn't exist for smartphone_accelerometer
Removing 6542d311-9d13-400a-8a3c-61f0d52ef6f5 as it doesn't exist for smartphone_accelerometer
Removing 41c392cb-3495-4138-a1b2-f056704f160c as it doesn't exist for smartphone_accelerometer
Removing eb5c7874-8961-4bac-b1cb-e35e4d715ca5 as it doesn't exist for smartphone_accelerometer
Removing 0b37bc64-905b-4234-af29-9760c9e16346 as it doesn't exist for smartphone_accelero

Removing cf49a3ae-8921-4264-99ee-a06d5f05575a as it doesn't exist for smartphone_accelerometer
Removing 99200aa0-4834-4f34-96d9-71c3c1b765b1 as it doesn't exist for smartphone_accelerometer
Removing f2a6831f-e712-4eb7-b150-ac44f520679c as it doesn't exist for smartphone_accelerometer
Removing fd3e05f6-8d9a-43c9-8a93-2231d341650d as it doesn't exist for smartphone_accelerometer
Removing 732aa52a-ce30-43d7-9fb0-647aeeb49957 as it doesn't exist for smartphone_accelerometer
Removing b09b5104-3a37-4819-b7f5-ba721d024722 as it doesn't exist for smartphone_accelerometer
Removing c0a5ca86-ac24-4d73-8623-c2923677c617 as it doesn't exist for smartphone_accelerometer
Removing f0911246-5bbd-4299-9ead-a60551b2ebfc as it doesn't exist for smartphone_accelerometer
Removing 18cd67ad-b280-4520-9582-d38b4f8004f8 as it doesn't exist for smartphone_accelerometer
Removing 59a2afdb-b324-421d-a0ba-e1b08dcd0585 as it doesn't exist for smartphone_accelerometer
Removing 1cc8c683-44a0-4fa8-9b00-06d86abc0c89 as i

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/d4474cb5-c1c2-497d-921e-3ec7abe02f86.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/58ea7790-a93c-4a9f-81c0-4757f237346e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/a4bf0da9-7116-4b57-ada2-d4e6fa116253.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/de88da3c-eff1-4827-ae3b-31472b96186f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/ed80c2a5-135e-4f90-9649-b5a6db5aa399.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/4e26e20d-344b-4bbf-9dbf-4a389c73784d.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_dat

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/33f5a031-43a8-496a-89ee-0b9d99019617.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/9dda3848-0758-4889-81ac-7a2d4fee50ef.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/d02209e2-ccb6-43ca-88a4-5053c51312f9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/9410e6a1-be87-4b03-be0d-fae812c52d91.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/33f5a031-43a8-496a-89ee-0b9d99019617.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/7e5db015-a0ad-4898-af48-8dbb927aad28.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/278622b7-49af-4730-8e5a-c7e9c91c49b9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/d417f853-2455-4038-9086-fe7832ff9010.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/39462bad-7b92-4b0b-865a-3d3091da34d2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/0db55f4b-d580-4e3c-a55c-8f76a5321af1.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/89094c0a-76f0-4b6c-8ff7-e3d869f86232.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartphone_accelerometer/d417f853-2455-4038-9086-fe7832ff9010.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_

...

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/63acb975-020e-408c-9887-6484817dd1d4.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/bb189dde-08f7-4678-8433-4716c6ffa8db.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/bb189dde-08f7-4678-8433-4716c6ffa8db.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/6ce4e0c5-a143-4b88-b2da-55e800df3939.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/6ce4e0c5-a143-4b88-b2da-55e800df3939.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/f6e2ff78-bca5-4a91-901a-4bbd1f7734f7.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_dat

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/48e84c8b-eeff-449f-894f-b8f32c573481.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/67b17ed4-530e-473a-b9c2-8328350d53a0.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/67b17ed4-530e-473a-b9c2-8328350d53a0.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/9588ee17-ba87-4f89-8fce-7be9032e0173.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/9588ee17-ba87-4f89-8fce-7be9032e0173.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/d1de0d41-67f2-420a-89bf-67625db6a202.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_dat

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/cf49a3ae-8921-4264-99ee-a06d5f05575a.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/1872cee9-3d53-4e8d-8d95-2145dd4e436e.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/1872cee9-3d53-4e8d-8d95-2145dd4e436e.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/99200aa0-4834-4f34-96d9-71c3c1b765b1.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/99200aa0-4834-4f34-96d9-71c3c1b765b1.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/b288f078-92bf-4907-ab75-6e594b92ecaf.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_dat

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/6c94b7d8-9be0-4212-80a6-800436b1fd74.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/f24c8fcb-1b16-497f-ac12-a938235cc43c.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/f24c8fcb-1b16-497f-ac12-a938235cc43c.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/3a18baf4-ddd8-48ad-8419-1a802eef4ecf.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/3a18baf4-ddd8-48ad-8419-1a802eef4ecf.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/80d3e287-4c41-4e6e-8ce6-9d1f74de9141.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_dat

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/8e93c916-02c4-441a-9505-d3a359105f2a.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/49f80736-6b50-44a6-a77b-9b1572334a8c.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/49f80736-6b50-44a6-a77b-9b1572334a8c.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/0a27e364-ead7-4d82-9e5d-e9538728f438.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartwatch_gyroscope/0a27e364-ead7-4d82-9e5d-e9538728f438.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data.wav_Z/smartwatch_gyroscope/ab76a089-f760-49b4-9351-4afda56abc2b.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_dat

# Tests & Drafts, back-up space that's not important, just notes

In [ ]:
# Change the default option to display all row with display(DF)
# pd.set_option('display.max_rows', 1000)

### Tried to do convolution instead of np.multiply and dot product to get the derivative

In [ ]:
df2 = pd.DataFrame(
    np.array(
        [
            [0.2, 1, 5, 9],
            [0.4, 2, 6, 10],
            [0.6, 3, 7, 11],
            [0.8, 4, 8, 12],
            [1, 5, 9, 13],
            [1.2, 6, 10, 14],
            [1.4, 7, 11, 15],
            [1.6, 7, 11, 15],
            [1.8, 8, 12, 16],
            [2, 9, 13, 16],
        ]
    ),
    columns=["Timestamp", "X", "Y", "Z"],
)
display(df2)
m = np.linspace(-3, 3, num=2 * 3 + 1)
display(m)

np.convolve(df2.loc[0:6, "X"], m)

### How to filter a DataFrame

In [ ]:
# Analysis of the data to find edge cases


# Create variable with TRUE if nationality is USA
dys = df_train_label["dyskinesia"] > 1

# Create variable with TRUE if age is greater than 50
tre = df_train_label["on_off"] > 0

# Select all cases where nationality is USA and age is greater than 50
df_train_label[dys & tre]

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)

In [ ]:
# df_zeros = pd.DataFrame([False,True,False,False,False,True,False]).astype(int)

df_zeros = np.array([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=bool)

display(df_zeros.astype(int))
count = 0
duration_threshold = 2
indices_list = []  # List of tuples
howmany = 0
for i in range(0, len(df_zeros)):
    if df_zeros[i] == 1:
        count = count + 1
        print("1 à lindex", i)
    else:
        if count >= duration_threshold:
            print("threshold atteint start ", start, " end at ", end)
            start = i - count
            end = i - 1
            indices_list.append((start, end))
            howmany = howmany + 1
            count = 0
        # if it doesn't reach the threshold, we change the 1 for 0 because we don't want to remove those
        elif count >= 1:
            print("Effacer les 1 de ", start, " a ", end)
            df_zeros[i - count : i] = [0] * (end - start)
            count = 0

display(df_zeros.astype(int))